# NASA EarthData Login (EDL)
### Authentication for data on-prem and in the cloud

> This tutorial assumes you already know the dataset's `short_name` or the dataset's `concept_id`.

In [ ]:
import earthaccess

auth = earthaccess.login()
print(f"Using earthaccess version {earthaccess.__version__}")
# are we authenticated

In [ ]:
auth = earthaccess.login(strategy="environment")

In [ ]:
auth = earthaccess.login(strategy="netrc")

In [ ]:
# using earthaccess authenticated session to fetch restricted files behind EDL
req_session = earthaccess.get_requests_https_session()

restricted_podaac_resource = "https://e4ftl01.cr.usgs.gov/ECOB/ECOSTRESS/ECO2LSTE.001/2022.07.13/ECOSTRESS_L2_LSTE_22783_013_20220713T101229_0601_02.h5"
restricted_gesdisc_resource = "https://data.gesdisc.earthdata.nasa.gov/data/MERGED_IR/GPM_MERGIR.1/1998/002/merg_1998010217_4km-pixel.nc4"
restricted_lpdaac_resource = "https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/ASTGTM.003/ASTGTMV003_N01E024_dem.tif"
# reading the first 100 bytes
data = req_session.get(restricted_lpdaac_resource, headers={"Range": "bytes=0-100"})
data.text

In [ ]:
# Using earthaccess to get fsspec sessions and open remote resources behind EDL as local files
fs = earthaccess.get_fsspec_https_session()

with fs.open(restricted_gesdisc_resource) as f:
    print(f.read(20))

## Authentication in the cloud

If the collection is a cloud-hosted collection we can print the `summary()` and get the endpouint to the S3 credentials. These S3 credentials are temporary and we can use them with third party libraries such as s3fs, boto3 or awscli.

In [ ]:
from pprint import pprint

query = earthaccess.collection_query()
# This always returns an array
datasets = query.cloud_hosted(True).keyword("GLAH14").get(2)

for dataset in datasets:
    pprint(dataset.summary())

In [ ]:
# C2153549818-NSIDC_CPRD
granules = earthaccess.granule_query().concept_id("C2153549818-NSIDC_CPRD").get(10)
# granules = earthaccess.granule_query().concept_id("C1000000461-NSIDC_ECS").get(10)
granules[0]

In [ ]:
file = earthaccess.download(granules[0:2], "data")

## Getting temporary S3 credentials

Some DAACs (ASF) require to accept EULAs to be able to access their S3 Buckets. 

In [ ]:
temp_s3_credentials = earthaccess.get_s3_credentials("ORNLDAAC")
# these credentials expire after 1 hour
temp_s3_credentials